#### IMPORTS

In [1]:
import numpy as np
import pandas as pd
import glob
import os

import geopandas as gpd
import xarray as xr
import rioxarray as rxr
from pyproj import CRS
from shapely.geometry import mapping

import climate_indices
from climate_indices import indices

#### SETTING PARAMETERS

In [2]:
############################################################################ SETTING PARAMETERS  for the SPI #################################################################

scale = 3
distribution = climate_indices.indices.Distribution.gamma   #Fixed
data_start_year = 1980
calibration_year_initial = 1980
calibration_year_final = 2023
periodicity = climate_indices.compute.Periodicity.monthly   #Fixed

######################################################################################### CRS ############################################################################

crs_project = CRS.from_epsg(4326) #WGS84

######################################################################################### INPUTS ############################################################################

ERA5_input_path = 'C:/ANIN/SPEI/data/ERA5_monthly.nc'
ERA5_daily_input_folder = 'C:/ANIN/SPEI/data/DAILY' 

######################################################################################### OUPUTS ############################################################################

path_out = "C:/ANIN/SPEI/outputs/"
SPEI_ouput_file = "SPEI3v2.nc"

#### MASK FUNCTIONS

In [3]:
# Load the shapefile
def load_shape_file(filepath):
    """Loads the shape file desired to mask a grid.
    Args:
        filepath: Path to *.shp file
    """
    shpfile = gpd.read_file(filepath)
    print("""Shapefile loaded. To prepare for masking, run the function
        `select_shape`.""")
    return shpfile

#Create the mask
def select_shape(shpfile):
    """Select the submask of interest from the shapefile.
    Args:
        shpfile: (*.shp) loaded through `load_shape_file`
        category: (str) header of shape file from which to filter shape.
            (Run print(shpfile) to see options)
        name: (str) name of shape relative to category.
           Returns:
        shapely polygon
    """

    col_code = 'ISO3_CODE'
    country_codes = ['ZAF', 'LSO', 'SWZ']

    # Extract the rows that have 'ZAF', 'LSO', or 'SWZ' in the 'SOV_A3' column
    selected_rows = shpfile[shpfile[col_code].isin(country_codes)]

    # Combine the selected polygons into a single polygon
    unioned_polygon = selected_rows.geometry.unary_union

    # Convert the unioned polygon to a geopandas dataframe with a single row
    mask_polygon = gpd.GeoDataFrame(geometry=[unioned_polygon])
    
    print("""Mask created.""")

    return mask_polygon

#### MASK LAYER

In [4]:
#Load de shp
shpfile = load_shape_file('C:/ANIN/clip_layer/CNTR_RG_01M_2020_4326.shp')

#Create the mask layer
mask_layer = select_shape(shpfile)

#Giving a CRS
mask_layer.crs = crs_project

Shapefile loaded. To prepare for masking, run the function
        `select_shape`.
Mask created.


#### VARIABLE PROCESSING

#### ERA5 TEMPERATURE DATA

In [5]:
daily_temp_files = glob.glob(os.path.join(ERA5_daily_input_folder, '*.nc'))

##### Temperatures

In [6]:
def getTemperaturesData(temp_data):
    """Process the data to get daily max, mean and min temperatures as input to the SPEI function
    Args:
        data: netcdf file with hourly temperature data

        Returns
        Three dataarray for Tmax, Tmin and Tmean with monthly values
    """  
    temp_data = (temp_data * 0.0008411010185231105) + 289.25556089480324 #Rescaling the values

    #Create 3 variables for daily tmax, tmin and tmean
    temp_tmax = temp_data.resample(time ='D').max()-273.15
    temp_tmin = temp_data.resample(time ='D').min()-273.15
    temp_tmean = temp_data.resample(time ='D').mean()-273.15

    # Resample data from daily into monthly. 
    temp_tmax = temp_tmax.resample(time ='M').mean()
    temp_tmin = temp_tmin.resample(time ='M').mean()
    temp_tmean = temp_tmean.resample(time ='M').mean()

    # Resample original data from hourly into monthly to have the structure
    temperatures = temp_data.resample(time='M').mean()

    #Putting all together
    temperatures['tmax'] = temp_tmax
    temperatures['tmin'] = temp_tmin
    temperatures['tmean'] = temp_tmean

    #Separate in variables
    tmax = temperatures['tmax']
    tmax = tmax.drop_vars(['tmin', 'tmean'])
    tmax = tmax.reindex(y=list(reversed(tmax['y'])))    # Reverse the Y dimension values to increasing values (This is an issue of ERA5 datasets and other climatic datasets)
    tmax.rio.write_crs(crs_project, inplace=True)

    tmean = temperatures['tmean']
    tmean = tmean.drop_vars(['tmin', 'tmax'])
    tmean = tmean.reindex(y=list(reversed(tmean['y']))) # Reverse the Y dimension values to increasing values (This is an issue of ERA5 datasets and other climatic datasets)
    tmean.rio.write_crs(crs_project, inplace=True)

    tmin = temperatures['tmin']
    tmin = tmin.drop_vars(['tmax', 'tmean'])
    tmin = tmin.reindex(y=list(reversed(tmin['y'])))    # Reverse the Y dimension values to increasing values (This is an issue of ERA5 datasets and other climatic datasets)
    tmin.rio.write_crs(crs_project, inplace=True)

    #Mask the country
    tmax_masked = tmax.rio.clip(mask_layer.geometry.apply(mapping), crs=mask_layer.crs, all_touched=True, from_disk=True).squeeze()
    tmean_masked = tmean.rio.clip(mask_layer.geometry.apply(mapping), crs=mask_layer.crs, all_touched=True, from_disk=True).squeeze()
    tmin_masked = tmin.rio.clip(mask_layer.geometry.apply(mapping), crs=mask_layer.crs, all_touched=True, from_disk=True).squeeze()

    return tmax_masked, tmean_masked, tmin_masked

In [7]:
tmax_list = []
tmin_list = []
tmean_list = []

#Applying the function to each one of the files with hourly data. Appending the result in separate variable lists
for file in daily_temp_files:
    data = rxr.open_rasterio(file, masked=True)
    tmax_masked, tmean_masked, tmin_masked = getTemperaturesData(data)
    tmax_list.append(tmax_masked)
    tmean_list.append(tmean_masked)
    tmin_list.append(tmin_masked)

#Creating an xarray for each temp variable
Tmax = xr.concat(tmax_list, dim='time')
Tmean = xr.concat(tmean_list, dim='time')
Tmin = xr.concat(tmin_list, dim='time')

#Changing time format
Tmax['time'] = Tmax['time'].astype('datetime64[ns]')
Tmean['time'] = Tmean['time'].astype('datetime64[ns]')
Tmin['time'] = Tmin['time'].astype('datetime64[ns]')

#Cleaning
Tmax = Tmax.drop_vars('tmax')
Tmean = Tmean.drop_vars('tmean')
Tmin = Tmin.drop_vars('tmin')

#### ERA5 DATA

In [8]:
#Loading the data
data = rxr.open_rasterio(ERA5_input_path, masked=True)
#Giving a CRS
data.rio.write_crs(crs_project, inplace=True)
data['time'] = data['time'].astype('datetime64[ns]')  # Change to datetime format to fix with te temp data ahead

data = data.assign_coords(time=pd.to_datetime(data.time.values) + pd.offsets.MonthEnd(1))   #Change the firt day of the month for the last day of the month
#data = data.sel(time=slice('1980', '1989'))

##### Precipitation

In [9]:
#Extract and give shape to the precipitation data
def getPrecipData(precip_data):
    """Process the data to get precipitation as input to the SPEI function
    Args:
        data: netcdf file with precip data

        Returns
        DataArrayGroupBy grouped over point (y and x coordinates)
    """
    num_days_month = precip_data.time.dt.days_in_month #Necessary to multiply the daily values of the mean to the "size" of the month

    precip = (precip_data * 2.908522800670776e-07) + 0.009530702520736942 #Rescaling the values
    precip = precip*1000*num_days_month  # The original units are meters, we change them to milimeters, and multiply by the days of the month
    
# Reverse the Y dimension values to increasing values (This is an issue of ERA5 datasets and other climatic datasets)
    precip = precip.rename({'y': 'lat', 'x':'lon'})       #Necessary step
    precip = precip.reindex(lat=list(reversed(precip['lat'])))
    precip = precip.rename({'lat': 'y', 'lon':'x'})

#Mask the country
    precip_masked = precip.rio.clip(mask_layer.geometry.apply(mapping), crs=mask_layer.crs, all_touched=True, from_disk=True).squeeze()

#Giving the appropriate shape to da data
    precip_grouped = precip_masked.stack(point=('y', 'x')).groupby('point')
    print("""Precipitation data is prepared to serve
        as input for the SPEI index.""")

    return precip_grouped


In [10]:
#Get precipitation data
precip_data = data['tp']
precips_mm = getPrecipData(precip_data)


Precipitation data is prepared to serve
        as input for the SPEI index.


##### Surface solar radiation downwards

In [11]:
def getSSRDdata(ssrd_data):
    """Process the data to get ssrd as input to the SPEI function
    Args:
        data: netcdf file with ssrd data

        Returns
        DataArray with ssrd data)
    """    
    
    ssrd = (ssrd_data * 401.7449529244808) + 20805007.127523538 #Rescaling the values
    ssrd = ssrd * pow(10, -6)   # The original units are J/m2, we change them to MJ/m2

# Reverse the Y dimension values to increasing values (This is an issue of ERA5 datasets and other climatic datasets)
    ssrd = ssrd.rename({'y': 'lat', 'x':'lon'})       #Necessary step
    ssrd = ssrd.reindex(lat=list(reversed(ssrd['lat'])))
    ssrd = ssrd.rename({'lat': 'y', 'lon':'x'})

#Mask the country
    ssrd_masked = ssrd.rio.clip(mask_layer.geometry.apply(mapping), crs=mask_layer.crs, all_touched=True, from_disk=True).squeeze()

    print("""ssrd data is prepared to serve
        as input for the SPEI index.""")

    return ssrd_masked

In [12]:
# Rn - net radiation at the crop surface MJ m-2 day-1
ssrd_data = data['ssrd']
Rn = getSSRDdata(ssrd_data)

ssrd data is prepared to serve
        as input for the SPEI index.


##### Dewpoint

In [13]:
def getD2Mdata(d2m_data):
    """Process the data to get dewpoint temperature as input to the SPEI function
    Args:
        data: netcdf file with ssrd data

        Returns
        DataArray with temp dewpoint data
    """    

    d2m = (d2m_data * 0.0005415367126581264) + 278.3172341144562 #Rescaling the values
    d2m = d2m - 273.15   # The original units are K, we change them to ºC

# Reverse the Y dimension values to increasing values (This is an issue of ERA5 datasets and other climatic datasets)
    d2m = d2m.rename({'y': 'lat', 'x':'lon'})       #Necessary step
    d2m = d2m.reindex(lat=list(reversed(d2m['lat'])))
    d2m = d2m.rename({'lat': 'y', 'lon':'x'})

#Mask the country
    d2m_masked = d2m.rio.clip(mask_layer.geometry.apply(mapping), crs=mask_layer.crs, all_touched=True, from_disk=True).squeeze()

    print("""Dewpoint data is prepared to serve
        as input for the SPEI index.""")

    return d2m_masked

In [14]:
#Dewpoint at 2 m height ºC
d2m_data = data['d2m']
Tdew = getD2Mdata(d2m_data)

Dewpoint data is prepared to serve
        as input for the SPEI index.


##### Atmospheric pressure

In [15]:
def getSPdata(sp_data):
    """Process the data to get atmospheric pressure as input to the SPEI function
    Args:
        data: netcdf file with ssrd data

        Returns
        DataArray with atmospheric pressure data
    """    
    sp = (sp_data * 0.4743678757267331) + 87188.76281606214 #Rescaling the values
    sp = sp * (pow(10, -3))   # The original units are Pa, we change them to KPa

# Reverse the Y dimension values to increasing values (This is an issue of ERA5 datasets and other climatic datasets)
    sp = sp.rename({'y': 'lat', 'x':'lon'})       #Necessary step
    sp = sp.reindex(lat=list(reversed(sp['lat'])))
    sp = sp.rename({'lat': 'y', 'lon':'x'})

#Mask the country
    sp_masked = sp.rio.clip(mask_layer.geometry.apply(mapping), crs=mask_layer.crs, all_touched=True, from_disk=True).squeeze()

    print("""atmospheric pressure data is prepared to serve
        as input for the SPEI index.""")

    return sp_masked

In [16]:
#atmospheric pressure kPa
sp_data = data['sp']
P = getSPdata(sp_data)

atmospheric pressure data is prepared to serve
        as input for the SPEI index.


##### Wind

In [17]:
def getWindData(u10, v10):
    """Process the data to get the wind component as input to the SPEI function
    Args:
        data: netcdf file with ssrd data

        Returns
        DataArray with wind data data
    """    
    u10 = (u10 * 0.00013396971091090765) - 0.6710555445783558 #Rescaling the values of u10
    v10 = (v10 * 0.00017109205086388073) + 1.4564011559399002 #Rescaling the values of v10

    u2 = ((u10**2) + (v10**2))**0.5  # Getting wind component

# Reverse the Y dimension values to increasing values (This is an issue of ERA5 datasets and other climatic datasets)
    u2 = u2.rename({'y': 'lat', 'x':'lon'})       #Necessary step
    u2 = u2.reindex(lat=list(reversed(u2['lat'])))
    u2 = u2.rename({'lat': 'y', 'lon':'x'})

#Mask the country
    u2_masked = u2.rio.clip(mask_layer.geometry.apply(mapping), crs=mask_layer.crs, all_touched=True, from_disk=True).squeeze()

    print("""Wind data is prepared to serve
        as input for the SPEI index.""")

    return u2_masked

In [18]:
#u2 wind speed at 2 m height m s-1
u10 = data['u10']
v10 = data['v10']

u2 = getWindData(u10, v10)

Wind data is prepared to serve
        as input for the SPEI index.


##### Soil heat flux density

In [19]:
# G -  soil heat flux density MJ m-2 day-1  Fixed value
G = 0

#### ERA5-land variables


In [20]:
#es saturation vapour pressure kPa
def get_es(Tmax, Tmin):
    e0Tmax = 0.6108 * np.exp((17.27*Tmax) / (Tmax + 237.3))
    e0Tmin = 0.6108 * np.exp((17.27*Tmin) / (Tmin + 237.3))

    es = (e0Tmax - e0Tmin) / 2

    return es

#ea actual vapour pressure kPa
def get_ea(Tdew):
    ea = 0.6108 * np.exp((17.27*Tdew) / (Tdew + 237.3))

    return ea

#slope vapour pressure curve kPa °C-1
def get_svpc(Tmean):
    svpc = (4098 * (0.6108 * np.exp((17.27*Tmean) / (Tmean + 237.3)))) / ((Tmean + 237.3)**2)

    return svpc

#Psychometric constant
def get_psi_cnt(P):
    Cp = 0.001013   #specific heat at constant pressure MJ kg-1 °C-1
    epsi = 0.622    #ratio molecular weight of water vapour/dry air
    lamb = 2.45     #latent heat of vaporization MJ kg-1

    psi_cnt = (Cp * P) / (epsi * lamb)

    return psi_cnt

# ET0 function
def get_pet_mm(svpc, Rn, G, psi_cnt, Tmean, u2, es, ea):
    pet_mm = (((0.408 * svpc) * (Rn - G)) + (psi_cnt * (900 / (Tmean + 273))) * u2 * (es - ea)) / (svpc + (psi_cnt * (1 + (0.34*u2))))
    
    return pet_mm

In [21]:
#Getting the components
es = get_es(Tmax, Tmin)
ea = get_ea(Tdew)
svpc = get_svpc(Tmean)
psi_cnt = get_psi_cnt(P)

#Calculation pet
pet_mm = get_pet_mm(svpc, Rn, G, psi_cnt, Tmean, u2, es, ea)

#Giving the appropriate shape to da data. Grouping
pet_mm_grouped = pet_mm.stack(point=('y', 'x')).groupby('point')

#### APPLY SPEI FUNCTION

In [22]:
#####https://github.com/monocongo/climate_indices
spei_values = xr.apply_ufunc(indices.spei,
                            precips_mm,
                            pet_mm_grouped, 
                            scale,
                            distribution,
                            periodicity,
                            data_start_year,
                            calibration_year_initial,
                            calibration_year_final
                            )                 

# Unstack the array back into original dimensions
spei_results = spei_values.unstack('point')         


#### EXPORTING

In [23]:
spei_results.to_netcdf(f'{path_out}{SPEI_ouput_file}')